### Seq2Seq모델 빌드를 위한 데이터 전처리

In [1]:
import os
import re
import json
import numpy as np
import pandas as pd
from typing import Union

from konlpy.tag import Okt

In [2]:
PATH = "./data/ChatBotData.csv_short"
VOCAB_PATH = "./data/vocabulary.txt"

In [3]:
# 데이터 읽어오는 함수 정의
def load_data(path: str):
    data_df = pd.read_csv(path, header=0, sep=",")
    question, answer = data_df['Q'].to_list(), data_df['A'].to_list()
    return question, answer

In [4]:
# 텍스트에 대해 불필요한 문자들을 제거하고 split한다.
def word_tokenizer(text_list: list) -> list:
    vocab = []
    
    for text in text_list:
        filter = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]" # filter 정의
        CHANGE_FILTER = re.compile(filter)

        cleaned_text = re.sub(CHANGE_FILTER, "", text) # filter에 부합하는 문자들을 제거하고
        text_list = cleaned_text.split() # 공백을 기준으로 리스트화 하고
        vocab.extend(text_list) # 사전에 추가한다.
    
    return vocab

In [5]:
# 형태소 분석 후 리스트에 담는다
def preprocess_like_morphlized(data: list):
    morph_analyzer = Okt()
    result_data = []
    
    for seq in data:
        morphed_data = " ".join(morph_analyzer.morphs(seq))
        result_data.append(morphed_data)
        
    return result_data

In [6]:
# 정수형 인코딩 된 형태의 사전을 리턴한다
def make_vocabulary(vocabulary_list: list):
    char2idx = {char : i for i, char in enumerate(vocabulary_list)}
    idx2char = {i : char for i, char in enumerate(vocabulary_list)}
    
    return char2idx, idx2char

In [7]:
inputs, outputs = load_data(PATH)

data = inputs + outputs
vocab2 = list(set(word_tokenizer(preprocess_like_morphlized(data)))) # 중복제거

In [8]:
PAD = "<PAD>" # padding token
SOS = "<SOS>" # start of sentence token
EOS = "<EOS>" # end of sentence token
OOV = "<OOV>" # out of vocabulary token

PAD_IDX = 0 # padding token index
SOS_IDX = 1 # start of sentence token index
EOS_IDX = 2 # end of sentence token index
OOV_IDX = 3 # out of vocabulary token index

# insert into vocabulary
vocab2.insert(PAD_IDX, PAD)
vocab2.insert(SOS_IDX, SOS)
vocab2.insert(EOS_IDX, EOS)
vocab2.insert(OOV_IDX, OOV)

In [9]:
with open(VOCAB_PATH, "w", encoding="utf-8") as f:
    for word in vocab2:
        f.write(word + "\n")

In [10]:
vocab_list = []

if os.path.exists(VOCAB_PATH):
    with open(VOCAB_PATH, "r", encoding="utf-8") as f:
        for vocab in f.readlines():
            vocab_list.append(vocab.replace("\n", ""))
else:
    print(f"Cannot read {VOCAB_PATH} file")

In [11]:
char2idx, idx2char = make_vocabulary(vocab_list)

In [12]:
char2idx

{'<PAD>': 0,
 '<SOS>': 1,
 '<EOS>': 2,
 '<OOV>': 3,
 '나오세요': 4,
 '다음': 5,
 '좋아요': 6,
 '하지': 7,
 '를': 8,
 '걸리겠어': 9,
 '돈': 10,
 '생각': 11,
 '불': 12,
 '선물': 13,
 '만': 14,
 '설득': 15,
 '사람': 16,
 '또': 17,
 '필요한': 18,
 '설움': 19,
 '어서': 20,
 '혼자': 21,
 '달': 22,
 '것': 23,
 '그': 24,
 '게': 25,
 '은': 26,
 '질린다': 27,
 '궁금해': 28,
 '자의': 29,
 '새': 30,
 '나': 31,
 '같아요': 32,
 '줄까': 33,
 '좋다': 34,
 '해보세요': 35,
 '가난한': 36,
 '승진': 37,
 '에': 38,
 '을': 39,
 '그럴': 40,
 '빠를수록': 41,
 '비싼데': 42,
 '있어도': 43,
 '가스': 44,
 '즐기세요': 45,
 '인데': 46,
 '생일': 47,
 '평소': 48,
 '나왔다': 49,
 '해봐요': 50,
 '거짓말': 51,
 '가상': 52,
 '비': 53,
 '바빠': 54,
 '교회': 55,
 '훈훈해': 56,
 '가': 57,
 '하세요': 58,
 '열': 59,
 '에는': 60,
 '돌아가서': 61,
 '다시': 62,
 '사세요': 63,
 '빨리': 64,
 '끄고': 65,
 '필요했던': 66,
 '좀': 67,
 '좋을까': 68,
 '잊고': 69,
 '안': 70,
 '난다': 71,
 '도': 72,
 '싶어': 73,
 '운동': 74,
 '전생': 75,
 '나라': 76,
 '나갔어': 77,
 '예요': 78,
 '뭘': 79,
 '해': 80,
 '구': 81,
 '남자친구': 82,
 '거': 83,
 '갔어': 84,
 '같아': 85,
 '적당히': 86,
 '결단': 87,
 '잘생겼어': 88,
 '출발': 8

In [13]:
idx2char

{0: '<PAD>',
 1: '<SOS>',
 2: '<EOS>',
 3: '<OOV>',
 4: '나오세요',
 5: '다음',
 6: '좋아요',
 7: '하지',
 8: '를',
 9: '걸리겠어',
 10: '돈',
 11: '생각',
 12: '불',
 13: '선물',
 14: '만',
 15: '설득',
 16: '사람',
 17: '또',
 18: '필요한',
 19: '설움',
 20: '어서',
 21: '혼자',
 22: '달',
 23: '것',
 24: '그',
 25: '게',
 26: '은',
 27: '질린다',
 28: '궁금해',
 29: '자의',
 30: '새',
 31: '나',
 32: '같아요',
 33: '줄까',
 34: '좋다',
 35: '해보세요',
 36: '가난한',
 37: '승진',
 38: '에',
 39: '을',
 40: '그럴',
 41: '빠를수록',
 42: '비싼데',
 43: '있어도',
 44: '가스',
 45: '즐기세요',
 46: '인데',
 47: '생일',
 48: '평소',
 49: '나왔다',
 50: '해봐요',
 51: '거짓말',
 52: '가상',
 53: '비',
 54: '바빠',
 55: '교회',
 56: '훈훈해',
 57: '가',
 58: '하세요',
 59: '열',
 60: '에는',
 61: '돌아가서',
 62: '다시',
 63: '사세요',
 64: '빨리',
 65: '끄고',
 66: '필요했던',
 67: '좀',
 68: '좋을까',
 69: '잊고',
 70: '안',
 71: '난다',
 72: '도',
 73: '싶어',
 74: '운동',
 75: '전생',
 76: '나라',
 77: '나갔어',
 78: '예요',
 79: '뭘',
 80: '해',
 81: '구',
 82: '남자친구',
 83: '거',
 84: '갔어',
 85: '같아',
 86: '적당히',
 87: '결단',
 88: '잘생겼어',
 89: '출발

### 인코더, 디코더 용 데이터 구성

In [68]:
filter = "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"
CHANGE_FILTER = re.compile(filter)
MAX_LEN = 25

def make_sequence(value, dictionary, tokenize_as_morph = False, padding = "pre", position = ""):
    if not tokenize_as_morph: # value 안의 각 문장들이 형태소 분석이 되어있지 않은 상태라면
        value = preprocess_like_morphlized(value) # 위에서 정의해둔 함수로 형태소 분석을 진행한다.
    
    padded_sequence = [] # 패딩된 각 문장들을 담을 빈 리스트
    for seq in value: # value안의 각 문장들을 순회하면서
        temp_list = [] # zero padding, 인코딩된 문장 리스트를 담을 빈 리스트
        seq_idx = [] # 문장의 각 단어가 정수형 인코딩 된 후 그 정수를 담을 빈 리스트
        
        seq = re.sub(CHANGE_FILTER, "", seq) # 불필요한 문자들을 제거하고
        
        # 문장 안에 있는 각 단어에 대한 정수형 인코딩 시작
        for word in seq.split(): # 문장을 띄어쓰기 단위로 생성한 리스트의 각 단어들을 순회하면서
            if dictionary.get(word) is not None: # 만약 그 단어가 사전에 존재하는 단어라면
                seq_idx.append(dictionary[word]) # 사전에 할당되어 있는 숫자를 seq_idx에 append하고
            else: # 없는 단어라면
                seq_idx.append(dictionary[OOV]) # unknown 토큰을 append한다.
        # 문장에 대한 정수형 인코딩 끝

        # zero padding 시작
        if len(seq_idx) > MAX_LEN: # 만약 seq_idx의 길이가 MAX_LEN보다 길다면
            seq_idx = seq_idx[:MAX_LEN] # 최대 길이에서 자른다.
        
        # encoder input, encoder output, encoder target의 상황에 따라, 
        if position == "encoder": # encoder용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx))*[dictionary[PAD]] # 최대 길이 - index의 길이만큼 0으로 이뤄진 패딩 리스트를 생성한다.
        
            if padding == "pre": # pre padding이라면
                temp_list.extend(zero_padding) # zero padding값을 먼저 extend하고
                temp_list.extend(seq_idx)
            else: # 아니라면
                temp_list.extend(seq_idx)
                temp_list.extend(zero_padding) # zero padding 값을 나중에 extend한다.
        elif position == "decoder output": # decoder output용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx) - 1)*[dictionary[PAD]] # SOS토큰을 고려해서 1을 추가적으로 빼준다.
        
            if padding == "pre": # pre-padding 이라면
                temp_list.append(dictionary[SOS]) # SOS토큰을 먼저 append 하고
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend한다.
            else: # 아니라면
                temp_list.append(dictionary[SOS]) # SOS토큰을 먼저 append 하고
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend한 후
                temp_list.extend(zero_padding) # zero padding 값을 extend 한다.
        else: # decoder target용 데이터라면
            zero_padding = (MAX_LEN - len(seq_idx) - 1)*[dictionary[PAD]] # EOS토큰을 고려해서 1을 추가적으로 빼준다.
        
            if padding == "pre":  #pre-padding이라면
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend 하고
                temp_list.append(dictionary[EOS]) # EOS토큰을 append한다.
            else: # 아니라면
                temp_list.extend(seq_idx) # 정수형 인코딩된 문장 리스트를 extend 하고
                temp_list.extend(zero_padding) # zero padding 값을 extend 한 후
                temp_list.append(dictionary[EOS]) # EOS토큰을 append한다.
            
            
        padded_sequence.append(temp_list) # 패딩이 완료된 각 요소는 전체 요소를 담을 list에 append한다.
    
    return np.array(padded_sequence)

In [69]:
encoder_input = make_sequence(inputs, char2idx, position="encoder")
decoder_output = make_sequence(inputs, char2idx, position="decoder output")
decoder_target = make_sequence(inputs, char2idx, position="decoder target")

In [70]:
data_configs = {
    "char2idx" : char2idx,
    "idx2char" : idx2char,
    'vocab_size' : len(char2idx),
    'pad_symbol' : PAD,
    'sos_symbol' : SOS,
    'eos_symbol' : EOS,
    'ovv_symbol' : OOV
}

In [75]:
TRAIN_INPUTS = "./data/train_inputs.npy"
TRAIN_OUTPUTS = "./data/train_outputs.npy"
TRAIN_TARGETS = "./data/train_targets.npy"
DATA_CONFIGS = "./data/data_config.json"

In [73]:
np.save(TRAIN_INPUTS, encoder_input)
np.save(TRAIN_OUTPUTS, decoder_output)
np.save(TRAIN_TARGETS, decoder_target)

In [78]:
json.dump(data_configs, open(DATA_CONFIGS, 'w'))

In [79]:
! git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
